In [2]:
import sys
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
from fileutils import DataLoader as DataLoader
from fileutils import paths
from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
from clfutils import FitModels
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, hamming_loss
import os
import multiprocessing
import psutil
import pickle
from collections import defaultdict
import itertools

import os
import pandas as pd
import numpy as np
import pickle as pickle
############################################


In [10]:
## functions

def load_pickled_in_filename(file):
    # load a simple pickled file and return it. its a bit different to the method used for the dictionary as this
    # is pure Python 3.x
    pickle_in = open(file, 'rb')
    return pickle.load(pickle_in)

def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file


def forwardDates(list_of_keys, current_date):
    """
    return all the forward looking dates for each idxKey we use for training

    :param list_of_keys: dates i have model dates for out of sample
    :param current_date: current model date
    :return: forward dates for applying the fitted model
    """
    lookAheadKeys = sorted(i for i in list_of_keys if i > current_date)
    return lookAheadKeys


def evaluate_predictions(y_true, y_preds):
    """
    Performs evaluation comparison on y_true labels vs. y_pred labels
    on a classification.
    """
    accuracy = accuracy_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds, average='weighted')
    precision_macro = precision_score(y_true, y_preds, average='macro')
    precision_micro = precision_score(y_true, y_preds, average='micro')
    recall = recall_score(y_true, y_preds, average='weighted')
    f1_weighted = f1_score(y_true, y_preds, average='weighted')
    f1_macro = f1_score(y_true, y_preds, average='macro')
    f1_micro = f1_score(y_true, y_preds, average='micro')
    hamming_loss_value = hamming_loss(y_true, y_preds)

    metric_dict = {"accuracy": round(accuracy, 2),
                   "precision": round(precision, 2),
                   "recall": round(recall, 2),
                   "f1- weighted": round(f1_weighted, 2),
                   "f1- micro": round(f1_micro, 2),
                   "f1- macro": round(f1_macro, 2),
                   "Hamming Loss": round(hamming_loss_value, 2)}
    print(f"Acc: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score weighted: {f1_weighted:.2f}")
    print(f"F1 score macro: {f1_macro:.2f}")
    print(f"F1 score micro: {f1_micro:.2f}")
    print(f"Hamming Loss Value: {hamming_loss_value:.2f}")

    return metric_dict


def paths(path_name):
    '''
    :param path_name: short cut name for paths
    :return: path URL
    '''
    pathDict = {'experiments': '/media/ak/DataOnly/SecondAlternativeExperimentPath/',
                'symbols_features': '/media/ak/DataOnly/SymbolFeatureDirectories/',
                'main': '/media/ak/DataOnly/'}
    return pathDict[path_name]

In [9]:
def concatenate_all_oos_per_label(dict_list):
    list_of_dfs =[pd.DataFrame(l) for l in dict_list]
    combined_results =pd.concat([pd.DataFrame(l) for l in dict_list], axis=1)
    combined_results
    return combined_results
    
def create_mean_list_w_correct_label(df, label_string):
    df = pd.DataFrame(df.mean(axis=1))
    df =  df.rename(columns ={df.columns[0]: label_string}, inplace = True)
    return df

In [13]:
mainPath= paths('main')

In [ ]:
#  import numpy as np
# ans = {label : {metric : np.mean([value[metric] for value in values.values()]) for metric in values[list(values)[0]].keys()} for label, values in df_w_Data.items()}

In [20]:
symbol ='UU.L'
symbolData  = DataLoader(mainPath, str(symbol))
fittedModelsPath = os.path.join(mainPath, "ExperimentCommonLocs/FittedModels")
oosPredictionsPath = os.path.join(mainPath, "ExperimentCommonLocs/OOSPredictions")

symbols = sorted(os.listdir(paths('symbols_features')))
print(symbols)
alternate_labels_nos = [ 1, 2, 3, 4, 5, 6, 7]  # we have 7 alternative data types

['AAL.L', 'APF.L', 'AV.L', 'AZN.L', 'BARC.L', 'BATS.L', 'BLT.L', 'CCL.L', 'CEY.L', 'CNA.L', 'CPG.L', 'DGE.L', 'HSBA.L', 'IOG.L', 'ITV.L', 'KGF.L', 'LAND.L', 'LGEN.L', 'LLOY.L', 'MAB.L', 'MKS.L', 'NG.L', 'PRU.L', 'PSON.L', 'RB.L', 'RBS.L', 'RDSa.L', 'RDSb.L', 'REL.L', 'RR.L', 'RSA.L', 'RTO.L', 'SDR.L', 'SGE.L', 'SHP.L', 'SMIN.L', 'SPT.L', 'STAN.L', 'TSCO.L', 'ULVR.L', 'UU.L', 'VOD.L', 'WPP.L']


In [38]:
hmm_model_dates = list()
best_svc_dict = dict()
pickled_models = [f for f in os.listdir(fittedModelsPath) if str(symbol) in f]  # list of all the pickled models

for pickled_model in pickled_models: # pick a pickled model

    model_date = (pickled_model.split("_")[1])  # load an HMM date model
    model_path = os.path.join(fittedModelsPath, pickled_model)
    print(model_path)
    hmm_model_dates.append(pickled_model.split("_")[1])
    print(os.path.isfile(model_path))
    best_svc = open_pickle_filepath(model_path)
    best_svc_dict[model_date] = sorted(list(best_svc[symbol].keys())) 

/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20180309_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20180321_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20180412_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170628_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170627_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170609_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170921_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170712_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170802_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20170714_4_SingleKernelSVC.pkl
True
/media/ak/DataOnly/ExperimentCommonLocs/FittedModels/UU.L_20

In [32]:
sorted(list(best_svc[symbol].keys())) # these are really really labels and hmm features that the model has been trained so these are "in sample dates"
# what are these dates?

['20170626',
 '20170627',
 '20170628',
 '20170630',
 '20170703',
 '20170704',
 '20170706',
 '20170707',
 '20170710',
 '20170711',
 '20170712',
 '20170714',
 '20170720',
 '20170721',
 '20170724',
 '20170728',
 '20170731',
 '20170802',
 '20170803']

In [84]:
for hmm_idx, hmm_date in enumerate(symbolData.hmm_dates_list):
    symbolData.hmm_model_date_feature_list_filepaths(hmm_idx)

TypeError: join() argument must be str or bytes, not 'int'

In [48]:
hmm_feature_path= symbolData.symbol_features_path

In [49]:
hmm_dates= sorted(os.listdir(hmm_feature_dates))

In [71]:
idx=0 # hmm feature model index -->will need a loop here
hmm_feature_date_path= os.path.join(hmm_feature_path, hmm_dates[idx]) #pick the path
print(hmm_feature_date_path)


/media/ak/DataOnly/SymbolFeatureDirectories/UU.L/MODEL_BASED/20170601


In [82]:
symbol_feature_paths = {feature_file.split("_")[5]:os.path.join(hmm_feature_date_path, feature_file) for f, feature_file in enumerate(sorted(os.listdir(hmm_feature_date_path)))}
keys = sorted(list(symbol_feature_paths.keys()))
label_idx =1
for key in keys:
    labels_file_path = os.path.join(symbolData.symbol_specific_label_path(label_idx), key + ".csv")
    if os.path.isfile(labels_file_path):
        print("can train")
        print(labels_file_path)
    else:
        print("#################### Problem")
#         print(labels_file_path)


can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170602.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170605.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170606.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170607.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170608.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170609.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170612.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170613.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170614.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170615.csv
can train
/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne/UU.L/20170616.csv
can train
/media/ak/DataOnly/ExperimentComm

In [67]:
symbol_feature_paths

{'20170602': '/media/ak/DataOnly/SymbolFeatureDirectories/UU.L/MODEL_BASED/20170601/UU.L_3_states_features_date:_20180420_now:_20190109_.pickle'}

In [55]:
main_labels_path = os.path.join(mainPath, 'ExperimentCommonLocs')

In [57]:
os.listdir(main_labels_path)

['LabelsAlternateThree',
 'LabelsAlternateFour',
 'LabelsAlternateSeven',
 'FittedModels',
 'LabelsAlternateOne',
 'LabelsAlternateSix',
 'OOSPredictions',
 'LabelsAlternateTwo',
 'JointLocationsDicts',
 'LabelsAlternateFive']

In [85]:
import multiprocessing

In [86]:
multiprocessing.Semaphore(multiprocessing.cpu_count())

<Semaphore(value=16)>